<!-- 파일 경로: C:\githome\9-11week_py_statistics\43_credit_card_summary.ipynb -->

# 신용카드 채무 불이행 고객 예측 (분류 문제)

**학습 목표:**
1. 실제 캐글 데이터를 사용하여 데이터 전처리 및 탐색적 데이터 분석(EDA)을 수행합니다.
2. 신용카드 고객 데이터를 기반으로 다음 달 채무 불이행(연체) 여부를 예측하는 분류 모델링 과정을 이해합니다.
3. 피처 간의 상관관계를 히트맵으로 시각화하고 분석합니다.

## 1. 데이터셋 소개

대만의 신용카드사 고객 30,000명의 인구통계 정보, 신용 한도, 과거 상환 내역 등을 바탕으로 다음 달 연체 여부를 예측하는 문제입니다.

- **데이터 출처:** [Kaggle - Default of Credit Card Clients Dataset](https://www.kaggle.com/datasets/uciml/default-of-credit-card-clients-dataset)

### 주요 피처(Feature) 설명

- `LIMIT_BAL`: 신용 한도액 (대만 달러)
- `SEX`: 성별 (1: 남성, 2: 여성)
- `EDUCATION`: 교육 수준 (1: 대학원, 2: 대학교, 3: 고등학교, 4: 기타)
- `MARRIAGE`: 결혼 상태 (1: 기혼, 2: 미혼, 3: 기타)
- `AGE`: 나이
- `PAY_1` ~ `PAY_6`: 과거 6개월간(4월~9월)의 상환 상태 (-1: 정시 상환, 1: 1개월 연체, 2: 2개월 연체, ...)
- `BILL_AMT1` ~ `BILL_AMT6`: 과거 6개월간의 청구 금액
- `PAY_AMT1` ~ `PAY_AMT6`: 과거 6개월간의 지불 금액
- **`default` (Target)**: 다음 달 채무 불이행 여부 (1: 연체, 0: 정상 상환)

In [ ]:
# 필요한 라이브러리를 임포트합니다.
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# CSV 파일을 읽어 DataFrame으로 변환합니다.
df = pd.read_csv('./data/UCI_Credit_Card.csv')

# 불필요한 'ID' 컬럼을 삭제합니다.
card_df = df.drop('ID', axis=1)
card_df.head(3)

## 2. 데이터 전처리 및 탐색

모델링에 적합한 형태로 데이터를 가공하고, 데이터의 특성을 파악합니다.

In [ ]:
# 컬럼 이름을 더 직관적으로 변경합니다.
# PAY_0 -> PAY_1: 9월 상환 상태
# default.payment.next.month -> default: 타겟 변수
card_df = card_df.rename(columns={'PAY_0':'PAY_1', 'default.payment.next.month':'default'})

# 타겟 변수와 피처 데이터를 분리합니다.
y_target = card_df['default']
X_features = card_df.drop('default', axis=1)

# 데이터의 기본 정보를 확인합니다. (결측치 없음)
X_features.info()

### 상관관계 분석

피처들 간의 상관관계를 히트맵으로 시각화하여 변수 간의 관계를 파악합니다. 이는 다중공선성 문제(상관관계가 매우 높은 피처들이 모델 성능에 악영향을 주는 현상)를 확인하는 데 도움이 됩니다.

In [ ]:
# 피처 간의 상관계수를 계산합니다.
corr = X_features.corr()

# 히트맵으로 시각화합니다.
plt.figure(figsize=(14,14))
sns.heatmap(corr, annot=True, fmt='.1g') # annot=True: 값 표시, fmt='.1g': 소수점 형식
plt.title('Correlation Heatmap of Credit Card Features')
plt.show()

# **분석 결과:**
# - `BILL_AMT1` ~ `BILL_AMT6` (청구 금액) 피처들 간의 상관관계가 매우 높게 나타납니다.
# - `PAY_1` ~ `PAY_6` (상환 상태) 피처들 간에도 양의 상관관계가 보입니다.
# - 이렇게 상관관계가 높은 피처들은 모델 학습 시 다중공선성 문제를 일으킬 수 있으므로, PCA를 이용한 차원 축소나 일부 피처를 선택/제거하는 방법을 고려해볼 수 있습니다.

## 3. 향후 진행 방향

이 노트북은 데이터 로딩과 기본적인 탐색까지만 다루었습니다. 실제 예측 모델을 만들기 위해서는 다음과 같은 과정이 추가로 필요합니다.

1. **데이터 분할**: 학습 데이터와 테스트 데이터를 분리합니다. (`train_test_split`)
2. **모델 학습**: 로지스틱 회귀, 랜덤 포레스트, XGBoost, LightGBM 등 다양한 분류 모델을 학습시킵니다.
3. **모델 평가**: 정확도(Accuracy), 정밀도(Precision), 재현율(Recall), F1 스코어, ROC-AUC 등 분류 성능 지표를 사용하여 모델을 평가합니다.
4. **모델 튜닝**: 하이퍼파라미터 튜닝(GridSearchCV, RandomizedSearchCV)을 통해 모델의 성능을 최적화합니다.
5. **피처 엔지니어링**: 상관관계가 높은 `BILL_AMT`나 `PAY_AMT` 피처들을 PCA를 통해 새로운 피처로 만들거나, 일부를 제거하여 모델 성능을 개선할 수.